In [1]:
import deepspeed

/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

print("Allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")
print("Reserved:", torch.cuda.memory_reserved() / 1024**2, "MB")

Allocated: 0.0 MB
Reserved: 0.0 MB


In [62]:
torch.cuda.get_device_properties(0).total_memory/1e9

23.684841472

In [63]:
s=1024
b=8
h=512
l = 16
a=32
print(s*b*h*4*l/1e9)
print(s*s*b*a*l/1e9)

0.268435456
4.294967296


In [64]:
import importlib, os

from models.deepseek import model
importlib.reload(model)

from models.deepseek import train
importlib.reload(train)

from models.deepseek import data
importlib.reload(data)

<module 'models.deepseek.data' from '/home/ubuntu/projects/llm-models/llm-models/models/deepseek/data.py'>

In [65]:
scale = 2
model_args = model.ModelArgs(expert_type=model.ExpertType(1),
                             dim=128*scale, inter_dim=341*scale, moe_inter_dim=64*scale,
                             kv_lora_rank=32*scale, qk_nope_head_dim=8*scale, qk_rope_head_dim=4*scale, 
                             v_head_dim=8*scale, 
                             n_routed_experts=32, n_layers=4*scale,
                                original_seq_len=data.CONTEXT_LENGTH)

In [66]:
16*32

512

In [67]:
training_args = train.TrainingArgs(
    model_args=model_args,
    batch_size=8,
    gradient_accumulation_steps=16,
    max_steps=100000,
    learning_rate=3e-4,
    checkpoint_dir="./checkpoints",
    data_dir="./data"
)

In [36]:
block_test = model.Block(2, model_args, False)
block_test.to('cuda:0')

Block(
  (attn): MLA(
    (wq): ColumnParallelLinear()
    (wkv_a): Linear()
    (kv_norm): RMSNorm()
    (wkv_b): ColumnParallelLinear()
    (wo): RowParallelLinear()
  )
  (ffn): MoE(
    (experts): MoE(
      (deepspeed_moe): MOELayer(
        (gate): TopKGate(
          (wg): Linear(in_features=256, out_features=32, bias=False)
        )
        (experts): Experts(
          (deepspeed_experts): ModuleList(
            (0-31): 32 x OptimizedModule(
              (_orig_mod): Expert(
                (w1): Linear()
                (w2): Linear()
                (w3): Linear()
              )
            )
          )
        )
      )
    )
    (shared_experts): MLP(
      (w1): ColumnParallelLinear()
      (w2): RowParallelLinear()
      (w3): ColumnParallelLinear()
    )
  )
  (ffn_norm): RMSNorm()
  (attn_norm): RMSNorm()
)

In [11]:
x = torch.zeros((4, 1024, 256))

In [14]:
x[:, 1023, :] = torch.rand((4, 256))

In [49]:
freqs_cis = model.precompute_freqs_cis(model_args)
freqs_cis = freqs_cis.to(torch.device("cuda"))

In [48]:
freqs_cis.device

device(type='cpu')

In [44]:
next(block_test.parameters()).device

device(type='cuda', index=0)

In [50]:
deepspeed.init_distributed()

with torch.amp.autocast('cuda',enabled=True, dtype=torch.bfloat16):
    y = block_test(x.to(torch.bfloat16).to('cuda:0'), 0, freqs_cis[0:1024], None)

/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return torch._C._get_cublas_allow_tf32()


In [58]:
y[0, 1023, :]

tensor([ 0.0234,  0.5469,  0.9922,  0.9062,  0.3594,  0.2656,  0.7578,  0.8359,
         0.1152,  0.8672,  0.9219,  0.6211,  0.6211,  0.0635,  0.2832,  1.1172,
         0.9648,  0.2773,  0.3574,  0.3965,  0.3457,  1.0469,  0.2412,  0.7773,
         1.0078,  0.7656,  0.5391,  1.1016, -0.3652,  0.6172, -0.1128,  0.7148,
         0.0059,  0.7266,  0.6367,  0.6562,  0.0193,  0.6250,  0.7266, -0.0225,
         0.6406,  0.4570, -0.1494,  0.6914,  0.3008,  0.3145,  0.5195,  0.4180,
         0.1621,  0.0488,  0.7227,  0.2988,  0.1699,  0.6367,  0.3320,  0.8008,
         0.4004,  0.7344,  0.4180,  0.6016,  0.3203,  0.6641,  0.7734,  0.1758,
         0.2100, -0.0298,  0.3926,  0.8906,  0.9531,  0.8125,  0.4746,  0.6680,
        -0.0859,  0.3398,  1.0000,  0.1621,  1.2578,  0.3320, -0.1118,  0.0591,
         0.7891,  0.1992,  0.6328,  0.3887,  0.7852,  0.4258,  0.1836,  0.8242,
         0.6719,  0.6211,  0.1357,  0.9805,  0.6562,  0.6758,  0.2852,  0.2734,
         0.6680,  0.4492,  0.1738,  0.43

In [59]:
x[0, 1023, :]

tensor([0.1729, 0.6641, 0.9702, 0.9048, 0.4517, 0.3549, 0.7460, 0.7938, 0.1769,
        0.6664, 0.8328, 0.4378, 0.4451, 0.1477, 0.2874, 0.9345, 0.8567, 0.4763,
        0.2634, 0.2032, 0.3045, 0.9303, 0.1845, 0.6166, 0.8214, 0.8066, 0.5206,
        0.9673, 0.1881, 0.3923, 0.0729, 0.6627, 0.2906, 0.6477, 0.6979, 0.6133,
        0.0755, 0.7288, 0.8441, 0.2299, 0.7341, 0.6355, 0.1486, 0.6586, 0.1066,
        0.3158, 0.6002, 0.6302, 0.1090, 0.2748, 0.8199, 0.5266, 0.0368, 0.5260,
        0.2439, 0.7841, 0.2484, 0.6122, 0.5485, 0.6036, 0.4290, 0.7579, 0.8235,
        0.2082, 0.2843, 0.0399, 0.2664, 0.8832, 0.9056, 0.5424, 0.4724, 0.5618,
        0.0839, 0.2183, 0.9922, 0.2510, 0.8713, 0.5478, 0.0816, 0.0140, 0.6259,
        0.3892, 0.8191, 0.4008, 0.7595, 0.4725, 0.1607, 0.5988, 0.8287, 0.7761,
        0.1400, 0.9261, 0.8425, 0.6161, 0.4850, 0.0938, 0.6996, 0.6518, 0.3275,
        0.4365, 0.9037, 0.3988, 0.1001, 0.2778, 0.4252, 0.8300, 0.1605, 0.2528,
        0.5811, 0.5127, 0.4040, 0.4088, 

In [33]:
import torch.distributed as dist

os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12356'
dist.init_process_group(backend='nccl', rank=0, world_size=1)

In [68]:
trainer = train.Trainer(training_args)

Total parameters: 173,437,184
Trainable parameters: 173,437,184
Loading *tokenized* TinyStories dataset split from: './processed_tinystories/train'...


/home/ubuntu/projects/llm-models/llm-models/models/deepseek/train.py:206: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.args.use_amp


Tokenized dataset loaded. Total chunks: 459757
Loading *tokenized* TinyStories dataset split from: './processed_tinystories/validation'...
Tokenized dataset loaded. Total chunks: 4619


In [11]:
trainer.model

Transformer(
  (embed): ParallelEmbedding()
  (layers): ModuleList(
    (0): Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MLP(
        (w1): ColumnParallelLinear()
        (w2): RowParallelLinear()
        (w3): ColumnParallelLinear()
      )
      (ffn_norm): RMSNorm()
      (attn_norm): RMSNorm()
    )
    (1-7): 7 x Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MoE(
        (experts): MoE(
          (deepspeed_moe): MOELayer(
            (gate): TopKGate(
              (wg): Linear(in_features=256, out_features=32, bias=False)
            )
            (experts): Experts(
              (deepspeed_experts): ModuleList(
                (0-31): 32 x Optimiz

In [8]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   4501 MiB |   4501 MiB |   4501 MiB |      0 B   |
|       from large pool |   4278 MiB |   4278 MiB |   4278 MiB |      0 B   |
|       from small pool |    223 MiB |    223 MiB |    223 MiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |   4501 MiB |   4501 MiB |   4501 MiB |      0 B   |
|       from large pool |   4278 MiB |   4278 MiB |   4278 MiB |

In [33]:
import torch
import gc
import pandas as pd

def list_cuda_tensors(model):
    # 1. Create a lookup table: Tensor ID -> Name
    tensor_name_map = {}
    for name, param in model.named_parameters():
        tensor_name_map[id(param)] = f"Param: {name}"
    for name, buf in model.named_buffers():
        tensor_name_map[id(buf)] = f"Buffer: {name}"

    memsize = 0
    print(f"{'Module/Tensor Name':<50} | {'Size (MB)':<10} | {'Shape'}")
    print("-" * 80)

    info = []
    # 2. Iterate through objects in memory
    for obj in gc.get_objects():
        try:
            # Check if it's a tensor
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                if obj.is_cuda:
                    # Look up the name using the object's ID
                    obj_id = id(obj)
                    name = tensor_name_map.get(obj_id, "Unknown (Intermediate/Other)")
                    
                    size_mb = obj.element_size() * obj.nelement() / 1024**2
                    memsize += size_mb
                    
                    # print(f"{name:<50} | {size_mb:>10.2f} | {tuple(obj.shape)}")
                    info.append({'name': f"{name:<50}", 'size_mb': size_mb, 'shape': obj.shape})
        except Exception:
            pass
            
    print("-" * 80)
    # print(f"Total GPU Memory tracked: {memsize:.2f} MB")
    return pd.DataFrame(info)

# Usage:
# list_cuda_tensors(your_model_instance)
    
infos = list_cuda_tensors(trainer.model)

Module/Tensor Name                                 | Size (MB)  | Shape
--------------------------------------------------------------------------------


/tmp/ipykernel_14128/2704726906.py:22: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):


--------------------------------------------------------------------------------


In [34]:
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.max_columns', None)   # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full content of each cell
pd.set_option('display.width', None)         # Auto-detect terminal width

In [35]:
infos = infos.sort_values(['size_mb'], ascending=False).reset_index()

In [40]:
infos.head(40)['size_mb'].sum()/1e3

6.8838125

In [41]:
infos.iloc[40:200,:]['size_mb'].sum()/1e3

FloatTrueDiv(2*(IntTrueDiv(1024*s64*s83, 1048576)) + 2351.375, 1000.0)

In [42]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(infos.sort_values(['size_mb'], ascending=False).head(100))
infos.head(200)

,index,name,size_mb,shape
0,827,Unknown (Intermediate/Other),1600.0,"(8, 1024, 102400)"
1,23426,Unknown (Intermediate/Other),256.0,"(8192, 64, 256)"
2,27794,Unknown (Intermediate/Other),256.0,"(8192, 64, 256)"
3,30706,Unknown (Intermediate/Other),256.0,"(8192, 64, 256)"
4,29978,Unknown (Intermediate/Other),256.0,"(8192, 64, 256)"
5,29250,Unknown (Intermediate/Other),256.0,"(8192, 64, 256)"
6,28522,Unknown (Intermediate/Other),256.0,"(8192, 64, 256)"
7,21970,Unknown (Intermediate/Other),256.0,"(8192, 64, 256)"
8,32162,Unknown (Intermediate/Other),256.0,"(8192, 64, 256)"
9,27066,Unknown (Intermediate/Other),256.0,"(8192, 64, 256)"


In [30]:
infos.head(220)['size_mb'].sum()

np.float64(3588.0)

In [38]:
trainer_model = model.Transformer(trainer.args.model_args)

In [13]:
trainer_model = trainer.model

In [60]:
for step, batch in enumerate(trainer.train_loader):
    break
    
x, y = batch
x = x.to(trainer.device, non_blocking=True)
y = y.to(trainer.device, non_blocking=True)

NameError: name 'trainer' is not defined

In [13]:
len(trainer.train_loader)

57470

In [17]:
114940/8/3600


3.990972222222222

In [14]:
trainer_model.to(trainer.device)

Transformer(
  (embed): ParallelEmbedding()
  (layers): ModuleList(
    (0): Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MLP(
        (w1): ColumnParallelLinear()
        (w2): RowParallelLinear()
        (w3): ColumnParallelLinear()
      )
      (ffn_norm): RMSNorm()
      (attn_norm): RMSNorm()
    )
    (1-7): 7 x Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MoE(
        (experts): MoE(
          (deepspeed_moe): MOELayer(
            (gate): TopKGate(
              (wg): Linear(in_features=256, out_features=32, bias=False)
            )
            (experts): Experts(
              (deepspeed_experts): ModuleList(
                (0-31): 32 x Optimiz

In [15]:
logits = trainer_model(x)

kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 1, 8])
k_nope shape: torch.Size([8, 1024, 16, 16])
v shape: torch.Size([8, 1024, 16, 16])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 1, 8])
k_nope shape: torch.Size([8, 1024, 16, 16])
v shape: torch.Size([8, 1024, 16, 16])


/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return torch._C._get_cublas_allow_tf32()


kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 1, 8])
k_nope shape: torch.Size([8, 1024, 16, 16])
v shape: torch.Size([8, 1024, 16, 16])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 1, 8])
k_nope shape: torch.Size([8, 1024, 16, 16])
v shape: torch.Size([8, 1024, 16, 16])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 1, 8])
k_nope shape: torch.Size([8, 1024, 16, 16])
v shape: torch.Size([8, 1024, 16, 16])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 1, 8])
k_nope shape: torch.Size([8, 1024, 16, 16])
v shape: torch.Size([8, 1024, 16, 16])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 1, 8])
k_nope shape: torch.Size([8, 1024, 16, 16])
v shape: torch.Size([8, 1024, 16, 16])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 1, 8])
k_nope shape: torch.Size([8, 1024, 16, 16])
v shape: torch.Size([8, 1024, 16, 16])


In [32]:
16*32

512

In [37]:
trainer.model = trainer_model

In [22]:
from models.deepseek.model import MoE

In [39]:
moe_aux_loss = 0.0
for name, module in trainer.model.named_modules():
    if isinstance(module, MoE) \
        and hasattr(module, '_moe_aux_loss') \
            and module._moe_aux_loss is not None:
        print(type(module))
        break

<class 'models.deepseek.model.MoE'>


In [40]:
trainer.collect_moe_aux_loss()

tensor(5.4817, device='cuda:0', dtype=torch.float32, grad_fn=<AddBackward0>)

In [16]:
for k, v in trainer.model.named_parameters():
    print(v.grad)
    break

None


In [18]:
trainer.args.log_interval

10

In [16]:
# trainer.args.max_steps=100
import time
time.time()
from torch.profiler import profile, ProfilerActivity, record_function


In [17]:
trainer.get_lr(trainer.global_step)

0.0

In [18]:
import math

In [14]:
import torch

torch.autograd.set_detect_anomaly(True)

In [24]:
class ActivationGradMonitor:
    def __init__(self, model):
        self.hooks = []
        for name, module in model.named_modules():
            # Filter for layers you care about (e.g., Norm, Linear, Attention)
            # if isinstance(module, (torch.nn.Linear, torch.nn.LayerNorm)):
            if 'layers.7.attn.kv_norm' in name:
                hook = module.register_full_backward_hook(self.make_hook(name))
                self.hooks.append(hook)
            if 'layers.7.attn.kv_norm' in name:
                hook = module.register_forward_hook(self.make_forward_hook(name))
                self.hooks.append(hook)

    def make_hook(self, name):
        def hook(module, grad_input, grad_output):
            # grad_output[0] is the gradient of the loss w.r.t. the output of this layer
            g = grad_output[0]
            # print(g)
            if g is not None:
                # print(f'{name}: max: {torch.max(g)}, min: {torch.min(g)}')
                print(f"[{name}] Output Grad -> Min: {g.min():.4e} | Max: {g.max():.4e} | Mean: {g.mean():.4e}")
                if torch.isnan(g).any():
                    print(f"!!! NAN DETECTED IN {name} Output GRADIENT !!!")
                    
            g = grad_input[0]
            # print(g)
            if g is not None:
                # print(f'{name}: max: {torch.max(g)}, min: {torch.min(g)}')
                print(f"[{name}] Input Grad -> Min: {g.min():.4e} | Max: {g.max():.4e} | Mean: {g.mean():.4e}")
                if torch.isnan(g).any():
                    print(f"!!! NAN DETECTED IN {name} Input GRADIENT !!!")
                    
        return hook

    def make_forward_hook(self, name):
        def hook(module, act_input, act_output):
            # grad_output[0] is the gradient of the loss w.r.t. the output of this layer
            g = act_output[0]
            # print(g)
            if g is not None:
                # print(f'{name}: max: {torch.max(g)}, min: {torch.min(g)}')
                print(f"[{name}] Output Activation -> Min: {g.min():.4e} | Max: {g.max():.4e} | Mean: {g.mean():.4e}")
                if torch.isnan(g).any():
                    print(f"!!! NAN DETECTED IN {name} Output GRADIENT !!!")
                
                    
            g = act_input[0]
            # print(g)
            if g is not None and name != 'embed':
                # print(f'{name}: max: {torch.max(g)}, min: {torch.min(g)}')
                print(f"[{name}] Input Activation -> Min: {g.min():.4e} | Max: {g.max():.4e} | Mean: {g.mean():.4e}")
                if torch.isnan(g).any():
                    print(f"!!! NAN DETECTED IN {name} Input GRADIENT !!!")
                    
        return hook
        
    def remove(self):
        for h in self.hooks:
            h.remove()

In [25]:
monitor = ActivationGradMonitor(trainer.model)

In [23]:
monitor.remove()

In [27]:
trainer.collect_moe_aux_loss()

tensor(7., device='cuda:0', dtype=torch.float32, grad_fn=<AddBackward0>)

In [21]:
def test_train(backend='inductor', run_profile=False):
    
    print('compiling')
    # trainer.model.compile(backend=backend)
    print('compiled')
    running_loss = 0.0
    start_time = time.time()

    if run_profile:
        print('prof start')
        trainer.prof.start()
        print('prof started')
        
    for step, batch in enumerate(trainer.train_loader):
        # print(step)
        trainer.prof.step()

        # Update learning rate
        lr = trainer.get_lr(trainer.global_step)
        for param_group in trainer.optimizer.param_groups:
            param_group['lr'] = lr
        
        # Training step
        loss = trainer.train_step(batch)
        for k, v in trainer.model.named_parameters():
            if v.grad != None and torch.any(torch.isnan(v.grad)):
                return

        running_loss += loss
        
        # Update weights after accumulation steps
        if (step + 1) % trainer.args.gradient_accumulation_steps == 0:
            # Gradient clipping
            
            trainer.scaler.unscale_(trainer.optimizer)
            torch.nn.utils.clip_grad_norm_(
                trainer.model.parameters(),
                trainer.args.grad_clip
            )
            
            # Optimizer step
            with record_function("## optimizer ##"):
                trainer.scaler.step(trainer.optimizer)
                trainer.scaler.update()
                trainer.optimizer.zero_grad(set_to_none=True)
            
            trainer.global_step += 1
            trainer.tokens_seen += (
                trainer.args.batch_size * 
                trainer.args.gradient_accumulation_steps * 
                trainer.args.model_args.max_seq_len * 
                trainer.world_size
            )
            
            # Logging
            if trainer.global_step % trainer.args.log_interval == 0 and trainer.is_main_process:
                    
                avg_loss = running_loss / trainer.args.log_interval
                elapsed = time.time() - start_time
                tokens_per_sec = trainer.tokens_seen / elapsed
                
                print(f"Step {trainer.global_step} | "
                      f"Loss: {avg_loss:.4f} | "
                      f"LR: {lr:.2e} | "
                      f"Tokens/sec: {tokens_per_sec:.0f}")
                
                running_loss = 0.0

            if trainer.global_step +1 == 80:
                return

            # Evaluation
            if trainer.global_step % trainer.args.eval_interval == 0:
                val_loss = trainer.evaluate()
                if trainer.is_main_process:
                    print(f"Validation loss: {val_loss:.4f}")
                    
                    if val_loss < trainer.best_val_loss:
                        trainer.best_val_loss = val_loss
                        trainer.save_checkpoint('best')
            
            # Checkpointing
            if trainer.global_step % trainer.args.save_interval == 0 and trainer.is_main_process:
                trainer.save_checkpoint(f'step_{trainer.global_step}')
            
            # Check if training is complete
            if trainer.global_step >= trainer.args.max_steps:
                if trainer.is_main_process:
                    print("Training complete!")
                    trainer.save_checkpoint('final')
                return

    # trainer.prof.stop()


In [26]:
test_train()

compiling
compiled


/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/torch/_inductor/lowering.py:1988: UserWarning: Torchinductor does not support code generation for complex operators. Performance may be worse than eager.
  warnings.warn(
/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return torch._C._get_cublas_allow_tf32()
/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/torch/_inductor/compile_fx.py:312: U

[layers.7.attn.kv_norm] Output Activation -> Min: -3.7612e+00 | Max: 3.7781e+00 | Mean: -1.0149e-01
[layers.7.attn.kv_norm] Input Activation -> Min: -2.5248e+00 | Max: 2.5619e+00 | Mean: -4.2136e-02
aux_loss: 0.0
[layers.7.attn.kv_norm] Output Grad -> Min: -1.7549e-07 | Max: 1.5225e-07 | Mean: -1.3143e-10
[layers.7.attn.kv_norm] Input Grad -> Min: -2.7333e-07 | Max: 2.6467e-07 | Mean: -1.8453e-10
[layers.7.attn.kv_norm] Output Activation -> Min: -3.5647e+00 | Max: 3.6609e+00 | Mean: -1.7461e-02
[layers.7.attn.kv_norm] Input Activation -> Min: -2.4698e+00 | Max: 2.3791e+00 | Mean: -2.8858e-02
aux_loss: 0.0
[layers.7.attn.kv_norm] Output Grad -> Min: -1.8716e-07 | Max: 1.4387e-07 | Mean: -1.6755e-10
[layers.7.attn.kv_norm] Input Grad -> Min: -2.8112e-07 | Max: 2.4443e-07 | Mean: -2.2494e-10
[layers.7.attn.kv_norm] Output Activation -> Min: -3.7034e+00 | Max: 3.4278e+00 | Mean: -8.8665e-02
[layers.7.attn.kv_norm] Input Activation -> Min: -2.7927e+00 | Max: 2.7046e+00 | Mean: -4.7941e-03
a

KeyboardInterrupt: 

In [20]:
for k, v in trainer.model.named_parameters():
    if v.grad != None and torch.any(torch.isnan(v.grad)):
    # if v.grad != None:
        print(f'BAD:{k}, max: {torch.max(v.grad)}, min: {torch.min(v.grad)}')
    else:
        if v.grad != None:
            print(f'good:{k}, max: {torch.max(v.grad)}, min: {torch.min(v.grad)}')
        # print(v.grad)

BAD:embed.weight, max: nan, min: nan
good:layers.0.attn.wq.weight, max: 0.0, min: 0.0
BAD:layers.0.attn.wkv_a.weight, max: nan, min: nan
good:layers.0.attn.kv_norm.weight, max: 0.0, min: 0.0
good:layers.0.attn.wkv_b.weight, max: 0.0, min: 0.0
good:layers.0.attn.wo.weight, max: 0.0, min: 0.0
good:layers.0.ffn.w1.weight, max: 0.0, min: 0.0
good:layers.0.ffn.w2.weight, max: 0.0, min: 0.0
good:layers.0.ffn.w3.weight, max: 0.0, min: 0.0
good:layers.0.ffn_norm.weight, max: 0.0, min: 0.0
BAD:layers.0.attn_norm.weight, max: nan, min: nan
good:layers.1.attn.wq.weight, max: 0.0, min: 0.0
good:layers.1.attn.wkv_a.weight, max: 0.0, min: 0.0
good:layers.1.attn.kv_norm.weight, max: 0.0, min: 0.0
good:layers.1.attn.wkv_b.weight, max: 0.0, min: 0.0
good:layers.1.attn.wo.weight, max: 0.0, min: 0.0
good:layers.1.ffn.experts.deepspeed_moe.gate.wg.weight, max: 0.0, min: 0.0
good:layers.1.ffn.experts.deepspeed_moe.experts.deepspeed_experts.0._orig_mod.w1.weight, max: 0.0, min: 0.0
good:layers.1.ffn.experts

In [21]:
for step, batch in enumerate(trainer.train_loader):
    break

In [27]:
batch[1]

tensor([[ 1263,    13, 40355,  ...,   465,   835,   290],
        [  284, 13502,  2405,  ...,   635, 11040,    13],
        [  356,   460,   526,  ...,   257,  7812,  7838],
        ...,
        [ 2933,  1444,  5689,  ...,   262, 10481,    13],
        [ 2613,    13,  1375,  ...,   655,  1234,   319],
        [  290,  4966,   706,  ...,    13,  1119,  6807]])

In [17]:
for k, v in trainer.model.named_parameters():
    # if v.grad != None and torch.any(torch.isnan(v.grad)):
    if v.grad != None:
        print(k)
        # print(v.grad)

embed.weight
layers.0.attn.wq.weight
layers.0.attn.wkv_a.weight
layers.0.attn.kv_norm.weight
layers.0.attn.wkv_b.weight
layers.0.attn.wo.weight
layers.0.ffn.w1.weight
layers.0.ffn.w2.weight
layers.0.ffn.w3.weight
layers.0.ffn_norm.weight
layers.0.attn_norm.weight
layers.1.attn.wq.weight
layers.1.attn.wkv_a.weight
layers.1.attn.kv_norm.weight
layers.1.attn.wkv_b.weight
layers.1.attn.wo.weight
layers.1.ffn.experts.deepspeed_moe.gate.wg.weight
layers.1.ffn.experts.deepspeed_moe.experts.deepspeed_experts.0._orig_mod.block.attn.wq.weight
layers.1.ffn.experts.deepspeed_moe.experts.deepspeed_experts.0._orig_mod.block.attn.wkv_a.weight
layers.1.ffn.experts.deepspeed_moe.experts.deepspeed_experts.0._orig_mod.block.attn.kv_norm.weight
layers.1.ffn.experts.deepspeed_moe.experts.deepspeed_experts.0._orig_mod.block.attn.wkv_b.weight
layers.1.ffn.experts.deepspeed_moe.experts.deepspeed_experts.0._orig_mod.block.attn.wo.weight
layers.1.ffn.experts.deepspeed_moe.experts.deepspeed_experts.0._orig_mod.b

In [ ]:
for k, v in trainer.model.named_parameters():
    # if v.grad != None and torch.any(torch.isnan(v.grad)):
    if v.grad != Nonen                                                                           :
        print(k)
        # print(v.grad)

In [18]:
for step, batch in enumerate(trainer.train_loader):
    # print(step)

    # Update learning rate
    lr = trainer.get_lr(trainer.global_step)
    for param_group in trainer.optimizer.param_groups:
        param_group['lr'] = lr
    
    # Training step
    loss = trainer.train_step(batch)
    break

In [21]:
for k, v in trainer.model.named_parameters():
    if torch.any(torch.isnan(v)):
        print(k)
        print(torch.isnan(v))
        print(v)
        # break

embed.weight
tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]], device='cuda:0')
Parameter containing:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       requires_grad=True)


In [13]:
trainer.prof.stop()

In [14]:
trainer.prof.export_chrome_trace("trace_deepspeed.json")

In [40]:
trainer.model.layers[1].ffn.experts.deepspeed_moe.experts.deepspeed_experts[0]._orig_mod.freqs_cis

tensor([[ 1.0000+0.0000j,  1.0000+0.0000j],
        [ 0.5403+0.8415j,  1.0000+0.0051j],
        [-0.4161+0.9093j,  0.9999+0.0102j],
        ...,
        [ 0.7412+0.6713j, -0.6445+0.7646j],
        [-0.1644+0.9864j, -0.6484+0.7613j],
        [-0.9188+0.3947j, -0.6523+0.7579j]], device='cuda:0')

In [14]:
MAX_NUM_OF_MEM_EVENTS_PER_SNAPSHOT: int = 100000

# Start recording memory snapshot history, initialized with a buffer
# capacity of 100,000 memory events, via the `max_entries` field.
torch.cuda.memory._record_memory_history(
   max_entries=MAX_NUM_OF_MEM_EVENTS_PER_SNAPSHOT
)

test_train('inductor', run_profile=False)

file_prefix = 'test'
try:
   torch.cuda.memory._dump_snapshot(f"{file_prefix}.pickle")
except Exception as e:
   logger.error(f"Failed to capture memory snapshot {e}")

# Stop recording memory snapshot history.
torch.cuda.memory._record_memory_history(enabled=None)

compiling
compiled
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [11]:
trainer_model = model.Transformer(trainer.args.model_args)

In [13]:
trainer_model = trainer.model

In [15]:
trainer_model.to(trainer.device)

Transformer(
  (embed): ParallelEmbedding()
  (layers): ModuleList(
    (0): Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MLP(
        (w1): ColumnParallelLinear()
        (w2): RowParallelLinear()
        (w3): ColumnParallelLinear()
      )
      (ffn_norm): RMSNorm()
      (attn_norm): RMSNorm()
    )
    (1-7): 7 x Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MoE(
        (experts): MoE(
          (deepspeed_moe): MOELayer(
            (gate): TopKGate(
              (wg): Linear(in_features=256, out_features=32, bias=False)
            )
            (experts): Experts(
              (deepspeed_experts): ModuleList(
                (0-31): 32 x Optimiz

In [69]:
for step, batch in enumerate(trainer.train_loader):
    break
    
x, y = batch
x = x.to(trainer.device, non_blocking=True)
y = y.to(trainer.device, non_blocking=True)

In [73]:
y.shape

torch.Size([8, 1024])

In [16]:
yy = trainer_model(x)

q_nope shape: torch.Size([8, 1024, 16, 16])
q_pe shape: torch.Size([8, 1024, 16, 8])
q_pe shape: torch.Size([8, 1024, 16, 8])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 8])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 1, 8])
k_nope shape: torch.Size([8, 1024, 16, 16])
v shape: torch.Size([8, 1024, 16, 16])
q_nope shape: torch.Size([8, 1024, 16, 16])
q_pe shape: torch.Size([8, 1024, 16, 8])
q_pe shape: torch.Size([8, 1024, 16, 8])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 8])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 1, 8])
k_nope shape: torch.Size([8, 1024, 16, 16])
v shape: torch.Size([8, 1024, 16, 16])


/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return torch._C._get_cublas_allow_tf32()


q_nope shape: torch.Size([8, 1024, 16, 16])
q_pe shape: torch.Size([8, 1024, 16, 8])
q_pe shape: torch.Size([8, 1024, 16, 8])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 8])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 1, 8])
k_nope shape: torch.Size([8, 1024, 16, 16])
v shape: torch.Size([8, 1024, 16, 16])
q_nope shape: torch.Size([8, 1024, 16, 16])
q_pe shape: torch.Size([8, 1024, 16, 8])
q_pe shape: torch.Size([8, 1024, 16, 8])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 8])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 1, 8])
k_nope shape: torch.Size([8, 1024, 16, 16])
v shape: torch.Size([8, 1024, 16, 16])
q_nope shape: torch.Size([8, 1024, 16, 16])
q_pe shape: torch.Size([8, 1024, 16, 8])
q_pe shape: torch.Size([8, 1024, 16, 8])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.Size([8, 1024, 8])
kv_lora shape: torch.Size([8, 1024, 64])
k_pe shape: torch.S

In [18]:
import numpy as np
np.exp(3)

np.float64(20.085536923187668)